### This script is for data retreival from DiffBind object

In [1]:
httr::set_config(httr::config(ssl_verifypeer = FALSE))
# set up the environment
# analysis tools
suppressPackageStartupMessages(library(DESeq2))
suppressPackageStartupMessages(library(DiffBind))

# table processing tools
suppressPackageStartupMessages(library(GenomicRanges))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(reshape))

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
Warning message:
“package ‘ggplot2’ was built under R version 4.2.3”
Warning message:
“package ‘tibble’ was built under R version 4.2.3”
Warning message:
“package ‘dplyr’ was built under R version 4.2.3”


In [2]:
outbase_dir = "/scratch/sclab/100322_ATAC"
# temporary output directories
temp_out = file.path(outbase_dir, "vscode_temp")
fileout_dir = file.path(temp_out, "data_new")
plotout_dir = file.path(temp_out, "figures_new")

### a. retrieve raw data by contrast

In [4]:
suffix <- "e80a"
dbObj<-dba.load(file=paste0("100322_atac_",suffix,"_postanalyze.readsInPeakNormed"), dir=file.path(outbase_dir, "ATACQC", "outputs"), pre='dba_', ext='RData')

In [5]:
dbObj

9 Samples, 119315 sites in matrix:
     ID Tissue Factor Replicate    Reads FRiP
1   wt1 retina     WT         1 40854780 0.44
2   wt2 retina     WT         2 48510284 0.46
3   wt3 retina     WT         3 41001963 0.34
4 ehet4 retina     AT         1 47819155 0.39
5 ehet5 retina     AT         2 37319506 0.46
6 ehet6 retina     AT         3 35847788 0.47
7 ehom1 retina     AA         1 41662566 0.43
8 ehom2 retina     AA         2 35254892 0.49
9 ehom3 retina     AA         3 39192437 0.43

Design: [~Factor] | 3 Contrasts:
  Factor Group Samples Group2 Samples2 DB.edgeR DB.DESeq2
1 Factor    WT       3     AT        3    29666     25171
2 Factor    WT       3     AA        3    76208     72436
3 Factor    AA       3     AT        3    39802     34478

In [9]:
# retrieve the consensus peakset
consensus.peaks <- dba.peakset(dbObj, bRetrieve=TRUE)
seqlevelsStyle(consensus.peaks) <- "UCSC"
# retain only canonical chromosome peaks
consensus.peaks = consensus.peaks[seqnames(consensus.peaks) %in% paste0("chr", c(1:22, "X", "Y"))]
# give each region a name
names(consensus.peaks) <- paste(suffix,"atac",1:length(consensus.peaks),sep="_")
consensus.peaks

GRanges object with 119315 ranges and 9 metadata columns:
                   seqnames          ranges strand |       wt1       wt2
                      <Rle>       <IRanges>  <Rle> | <numeric> <numeric>
       e80a_atac_1     chr1 3007274-3007674      * |   42.4317   73.2420
       e80a_atac_2     chr1 3094807-3095207      * |   40.4581   24.4140
       e80a_atac_3     chr1 3194302-3194702      * |   90.7841   68.5167
       e80a_atac_4     chr1 3197558-3197958      * |   58.2202   40.1650
       e80a_atac_5     chr1 3210204-3210604      * |   86.8369   81.1175
               ...      ...             ...    ... .       ...       ...
  e80a_atac_119311     chrY   809027-809427      * |   51.3127   61.4287
  e80a_atac_119312     chrY   872888-873288      * |   42.4317   47.2529
  e80a_atac_119313     chrY   897346-897746      * |   32.5639   41.7400
  e80a_atac_119314     chrY 1010188-1010588      * |  246.6958  283.5173
  e80a_atac_119315     chrY 1245604-1246004      * |  145.0572  19

In [10]:
saveRDS(consensus.peaks , file=file.path(fileout_dir, "atac_consensus_peakset", paste0("100322_atac_",suffix,"_readsInPeakNormed.consensus.centered.rds")))

In [48]:
# convert GRanges object to dataframe and keep the peak id as row name
tmp <- consensus.peaks %>% as_tibble()
tmp$name <- paste(suffix,"atac",rownames(tmp),sep="_")
tmp %>% column_to_rownames(var="name") %>%
write.table(file.path(fileout_dir, "atac_consensus_peakset", paste0("100322_atac_",suffix,"_readsInPeakNormed.consensus.centered.tsv")), quote=FALSE, sep="\t", col.names=TRUE, row.names=TRUE)
rm(tmp)

### extract differential accessibility analysis

In [70]:
#retrieve contrast table and keep only those compared with WT
atac.contrast <- dba.show(dbObj, bContrasts=T)
#atac.contrast <- atac.contrast[atac.contrast$Group=="WT",]
atac.contrast$Factor<- NULL
atac.contrast$Samples <- NULL
atac.contrast$Samples2 <- NULL
atac.contrast$Group <- tolower(atac.contrast[,"Group"])
atac.contrast$Group2 <- tolower(atac.contrast[,"Group2"])
atac.contrast$mut.loss <- rep("n.a.",time=nrow(atac.contrast))
atac.contrast$mut.gain <- rep("n.a.",time=nrow(atac.contrast))
# update the metaTable with the corresponding atac DB contrast dataset
for (row in c(1:nrow(atac.contrast))){
      atac.contrast[row,"mut.loss"] <- paste0(row,".A")
      atac.contrast[row,"mut.gain"] <- paste0(row,".B")
}
# add contrast name
atac.contrast$contrast <- paste0(atac.contrast$Group, "_vs_", atac.contrast$Group2)
atac.contrast$Group <- NULL
rownames(atac.contrast) <- NULL # get rid of the old rownames to avoid error
# use mutant as the rownames
#atac.contrast <- atac.contrast %>% column_to_rownames(var="Group2")
#print(atac.contrast)

In [71]:
atac.contrast

Group2,DB.edgeR,DB.DESeq2,mut.loss,mut.gain,contrast
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
at,29666,25171,1.A,1.B,wt_vs_at
aa,76208,72436,2.A,2.B,wt_vs_aa
at,39802,34478,3.A,3.B,aa_vs_at


In [23]:
retrieve_DB_res <- function(contrast, method_name, fdr.cutoff){
  if (method_name == "deseq2"){
    use_method<-DBA_DESEQ2
  }else if (method_name == "edger"){
    use_method<-DBA_EDGER
  }
  # extract the results from method_name and below the given FDR threshold
  # remove any no useful column containing all NAs
  DB.result <- dba.report(dbObj, contrast=contrast, method=use_method, th=fdr.cutoff) %>% 
              data.frame() %>% select(where(~!all(is.na(.)))) %>%
              makeGRangesFromDataFrame(keep.extra.columns=TRUE, ignore.strand=FALSE)
  return (DB.result)
}

In [72]:
# retreieve the deseq2 or edger contrast results table
all.DB.matrix <- lapply(c(1:nrow(atac.contrast)), function(x) retrieve_DB_res(x,"deseq2",1))
names(all.DB.matrix) <- atac.contrast$contrast

In [ ]:
all.DB.matrix

In [73]:
get_peak_lfc <- function(matrix.name){
        # retrieve factor names for the selected contrast
        mut <- str_split(matrix.name,"_",simplify=TRUE)[3]
        ctrl <- str_split(matrix.name,"_",simplify=TRUE)[1]
        print(paste("processing contrast",matrix.name, sep=" "))
        clean.matrix <- all.DB.matrix[[matrix.name]]
        # make sure it is UCSC convention
        seqlevelsStyle(clean.matrix) <- "UCSC"
        clean.matrix <- clean.matrix %>% data.frame() %>% filter_at(c(7,8), all_vars(.>=1))
        print(paste0("after removing regions with conc. lower than 1: ",nrow(clean.matrix)))
        gr <- clean.matrix %>%
                # for some matrix the log2FC is correct mut/wt but some are the reversed
                # if in the contrast meta table, the last column is wt vs mut, use log2FC = -Fold
                # if in the contrast meta table, the last column is mut vs wt, use log2FC = Fold
                dplyr::mutate(log2FC=-Fold) %>% 
                #filter(FDR < fdr.cutoff & Fold > fc.cutoff) %>%
                select(seqnames, start, end, strand, width, log2FC, p.value, FDR) %>%
                dplyr::arrange(seqnames, start)

        # rename lfc and fdr columns to make unique
        names(gr)[6:8] <- c(paste("atac",matrix.name,"lfc", sep="."), paste("atac",matrix.name,"p.value", sep="."), paste("atac",matrix.name,"fdr", sep="."))

        return(gr)
}

In [74]:
names(all.DB.matrix)

[1] "wt_vs_at" "wt_vs_aa" "aa_vs_at"

In [75]:
all.DB.lfc <- lapply(names(all.DB.matrix), function(x) get_peak_lfc(x))
names(all.DB.lfc) <- names(all.DB.matrix)

[1] "processing contrast wt_vs_at"
[1] "after removing regions with conc. lower than 1: 119315"
[1] "processing contrast wt_vs_aa"
[1] "after removing regions with conc. lower than 1: 119314"
[1] "processing contrast aa_vs_at"
[1] "after removing regions with conc. lower than 1: 119314"


In [76]:
names(all.DB.lfc)

[1] "wt_vs_at" "wt_vs_aa" "aa_vs_at"

In [77]:
sapply(names(all.DB.lfc), function(x) all.DB.lfc[[x]] %>% write.table(file=file.path(fileout_dir, "atac_diffbind_matrix", paste0(x,"_lfc.tsv")), quote=FALSE, sep="\t", col.names=TRUE, row.names=FALSE))

$wt_vs_at
NULL

$wt_vs_aa
NULL

$aa_vs_at
NULL

In [78]:
sapply(names(all.DB.matrix), function(x) all.DB.matrix[[x]] %>% data.frame() %>% write.table(file=file.path(fileout_dir, "atac_diffbind_matrix", paste0(x,"_lfc.unfiltered.tsv")), quote=FALSE, sep="\t", col.names=TRUE, row.names=FALSE))

$wt_vs_at
NULL

$wt_vs_aa
NULL

$aa_vs_at
NULL

In [79]:
# collect all chip differential binding tables in to one
e80a_atac_lfc <- all.DB.lfc[["wt_vs_at"]] %>% 
                dplyr::full_join(all.DB.lfc[["wt_vs_aa"]]) %>%
                dplyr::full_join(all.DB.lfc[["aa_vs_at"]]) %>%
                dplyr::arrange(seqnames, start)

Joining, by = c("seqnames", "start", "end", "strand", "width")
Joining, by = c("seqnames", "start", "end", "strand", "width")


In [39]:
e80a_atac_lfc %>% write.table(file.path(fileout_dir, "atac_diffbind_matrix", "e80a_atac_consensus_peaks_lfc.tsv"), quote=FALSE, sep="\t", col.names=TRUE, row.names=FALSE)

In [80]:
nrow(e80a_atac_lfc)

[1] 119315

In [59]:
# collect all chip differential binding tables in to one
k88n_atac_lfc <- all.DB.lfc[["wt_vs_nt"]] %>% 
                dplyr::full_join(all.DB.lfc[["wt_vs_nn"]]) %>%
                dplyr::full_join(all.DB.lfc[["wt_vs_ww"]]) %>%
                dplyr::full_join(all.DB.lfc[["nt_vs_ww"]]) %>%
                dplyr::full_join(all.DB.lfc[["ww_vs_nn"]]) %>%
                dplyr::arrange(seqnames, start)

Joining, by = c("seqnames", "start", "end", "strand", "width")
Joining, by = c("seqnames", "start", "end", "strand", "width")
Joining, by = c("seqnames", "start", "end", "strand", "width")
Joining, by = c("seqnames", "start", "end", "strand", "width")


In [60]:
k88n_atac_lfc %>% write.table(file.path(fileout_dir, "atac_diffbind_matrix", "k88n_atac_consensus_peaks_lfc.tsv"), quote=FALSE, sep="\t", col.names=TRUE, row.names=FALSE)

In [ ]:
nrow(k88n_atac_lfc)

In [61]:
sessionInfo()

R version 4.0.5 (2021-03-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Rocky Linux 8.7 (Green Obsidian)

Matrix products: default
BLAS/LAPACK: /ref/sclab/software/miniconda3/envs/specseq/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats4    stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] reshape_0.8.9               forcats_0.5.2              
 [3] stringr_1.4.1               dplyr_1.0.10               
 [5] purrr_0.3.4                 readr_2.1.3                
 [7] tidyr_1.2.1                 tibble_3.1.8               
 [9] ggplot2_3